In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# =====================================================
# 1. Device Configuration
# =====================================================
# Use GPU if available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# =====================================================
# 2. Hyperparameters
# =====================================================
# Batch size: number of samples processed before backprop
BATCH_SIZE = 64

# Number of complete passes over the training dataset
EPOCHS = 10

# Learning rate controls step size of optimizer
LEARNING_RATE = 0.001

# Fashion-MNIST has 10 clothing classes
NUM_CLASSES = 10


In [ ]:
# =====================================================
# 3. Data Preprocessing and Transformations
# =====================================================
# Convert images to PyTorch tensors and normalize pixel values
# Normalization helps stabilize and speed up training
transform = transforms.Compose([
    transforms.ToTensor(),                 # Converts [0,255] → [0,1]
    transforms.Normalize((0.5,), (0.5,))   # Normalizes grayscale channel
])


In [ ]:
# =====================================================
# 4. Load Fashion-MNIST Dataset
# =====================================================
# Training dataset (60,000 images)
train_dataset_full = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

# Test dataset (10,000 images)
test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True
)


100%|██████████| 26.4M/26.4M [00:02<00:00, 10.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 204kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.78MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 18.1MB/s]


In [ ]:
# =====================================================
# 5. Train–Validation Split
# =====================================================
# Split training data into 80% training and 20% validation
train_size = int(0.8 * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size

train_dataset, val_dataset = random_split(
    train_dataset_full, [train_size, val_size]
)

# DataLoaders handle batching, shuffling, and parallel loading
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# =====================================================
# 6. CNN Architecture Definition
# =====================================================
class FashionCNN(nn.Module):
    """
    CNN Architecture:
    Input (1×28×28)
    → Conv → ReLU → MaxPool
    → Conv → ReLU → MaxPool
    → Conv → ReLU → MaxPool
    → Fully Connected Layers
    """

    def __init__(self):
        super(FashionCNN, self).__init__()

        # -------- Feature Extraction Layers --------
        self.features = nn.Sequential(
            # First convolutional block
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            # Second convolutional block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Third convolutional block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # -------- Classification Layers --------
        # After 3 max-pooling layers:
        # Image size reduces from 28×28 → 14×14 → 7×7 → 3×3
        self.classifier = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, NUM_CLASSES)
        )

    def forward(self, x):
        # Forward pass through convolutional layers
        x = self.features(x)

        # Flatten feature maps into a 1D vector
        x = x.view(x.size(0), -1)

        # Forward pass through fully connected layers
        x = self.classifier(x)
        return x

# Instantiate model and move it to the chosen device
model = FashionCNN().to(device)
print(model)


FashionCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=1152, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
# =====================================================
# 7. Loss Function and Optimizer
# =====================================================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
# =====================================================
# 8. Training Loop
# =====================================================
def train_model():
    model.train()  # Set model to training mode

    for epoch in range(EPOCHS):
        running_loss = 0.0

        for images, labels in train_loader:
            # Move data to GPU/CPU
            images, labels = images.to(device), labels.to(device)

            # Clear old gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{EPOCHS}], "
              f"Training Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
# =====================================================
# 9. Evaluation Function
# =====================================================
def evaluate_model(loader):
    model.eval()  # Set model to evaluation mode

    y_true = []
    y_pred = []

    # Disable gradient computation
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)

            outputs = model(images)

            # Get class with highest probability
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="weighted"
    )
    cm = confusion_matrix(y_true, y_pred)

    return accuracy, precision, recall, f1, cm


In [ ]:
# =====================================================
# 10. Train the Model
# =====================================================
train_model()

Epoch [1/10], Training Loss: 0.5728
Epoch [2/10], Training Loss: 0.3498
Epoch [3/10], Training Loss: 0.2950
Epoch [4/10], Training Loss: 0.2607
Epoch [5/10], Training Loss: 0.2402
Epoch [6/10], Training Loss: 0.2136
Epoch [7/10], Training Loss: 0.1972
Epoch [8/10], Training Loss: 0.1799
Epoch [9/10], Training Loss: 0.1673
Epoch [10/10], Training Loss: 0.1539


In [ ]:
# =====================================================
# 11. Test Set Evaluation
# =====================================================
test_accuracy, test_precision, test_recall, test_f1, test_cm = evaluate_model(test_loader)

print("\nTest Set Performance:")
print(f"Accuracy : {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall   : {test_recall:.4f}")
print(f"F1-score : {test_f1:.4f}")

print("\nConfusion Matrix:")
print(test_cm)


Test Set Performance:
Accuracy : 0.9140
Precision: 0.9139
Recall   : 0.9140
F1-score : 0.9133

Confusion Matrix:
[[853   0  14  32   4   3  89   0   5   0]
 [  0 991   1   4   2   0   1   0   1   0]
 [ 14   1 859   8  68   0  50   0   0   0]
 [  7   7  10 928  30   0  15   0   1   2]
 [  0   1  28  17 916   0  36   0   2   0]
 [  0   0   0   0   0 973   0  20   0   7]
 [116   4  41  36  98   0 701   0   4   0]
 [  0   0   0   0   0   3   0 980   0  17]
 [  2   1   1   5   4   2   3   3 979   0]
 [  0   0   0   0   0   3   1  36   0 960]]
